In [1]:
!pip install tts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 566 kB 4.9 MB/s 
     |████████████████████████████████| 73 kB 3.0 MB/s 
     |████████████████████████████████| 183 kB 72.4 MB/s 
     |████████████████████████████████| 44 kB 3.8 MB/s 
     |████████████████████████████████| 577 kB 71.2 MB/s 
     |████████████████████████████████| 71 kB 9.5 MB/s 
     |████████████████████████████████| 287 kB 77.4 MB/s 
     |████████████████████████████████| 1.9 MB 55.5 MB/s 
     |████████████████████████████████| 1.4 MB 51.7 MB/s 
     |████████████████████████████████| 80 kB 13.1 MB/s 
     |████████████████████████████████| 47.4 MB 1.2 MB/s 
     |████████████████████████████████| 3.4 MB 64.0 MB/s 
     |████████████████████████████████| 292 kB 74.1 MB/s 
     |████████████████████████████████| 101 kB 16.3 MB/s 
     |████████████████████████████████| 15.2 MB 58.7 MB/s 
     |████████████████████████████████

In [ ]:
!tts

In [2]:
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.config.shared_configs import BaseAudioConfig
from TTS.utils.audio import AudioProcessor
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.tts.models.vits import Vits
from TTS.tts.configs.vits_config import VitsConfig
from trainer import Trainer, TrainerArgs
from TTS.tts.datasets import load_tts_samples

In [3]:
import os

In [4]:
!git clone https://github.com/hantswilliams/digitalclone-backend.git

Cloning into 'digitalclone-backend'...
remote: Enumerating objects: 345, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 345 (delta 82), reused 124 (delta 3), pack-reused 141
Receiving objects: 100% (345/345), 41.96 MiB | 45.42 MiB/s, done.
Resolving deltas: 100% (131/131), done.


In [5]:
!cd digitalclone-backend/ && git pull

Already up to date.


In [6]:
dataset_path = "/content/digitalclone-backend/aws_jobs_voiceclone/tts_tests/testdatasets/audiofiles/"
output_path = "/content/trainoutput"

In [7]:
# get list of files in audiofiles/wavs
audiofiles = os.listdir(dataset_path + "/wavs")
print('audiofiles...:', audiofiles)


audiofiles...: ['list1-q4_1664299449480.wav', 'list1-q2_1664299434662.wav', 'list1-q8_1664299479477.wav', 'list1-q1_1664299399500.wav', 'list1-q5_1664299457270.wav', 'list1-q7_1664299472544.wav', 'list1-q10_1664299494437.wav', 'list1-q6_1664299464874.wav', 'list1-q3_1664299442218.wav', 'list1-q9_1664299487431.wav']


In [8]:
# loop through each file in audiofiles/wavs and convert to 22050
for file in audiofiles:
    # get file name
    filename = file.split(".")[0]
    # get file extension
    fileext = file.split(".")[1]
    # create command to convert file to 22050
    command = "ffmpeg -i " + dataset_path + "/wavs/" + file + " -ar 16000 -ac 1 " + dataset_path + "/wavs/" + filename + "_mod." + fileext + " -y"
    # run command
    os.system(command)
    # print
    print("converted " + file + " to 16000")

converted list1-q4_1664299449480.wav to 16000
converted list1-q2_1664299434662.wav to 16000
converted list1-q8_1664299479477.wav to 16000
converted list1-q1_1664299399500.wav to 16000
converted list1-q5_1664299457270.wav to 16000
converted list1-q7_1664299472544.wav to 16000
converted list1-q10_1664299494437.wav to 16000
converted list1-q6_1664299464874.wav to 16000
converted list1-q3_1664299442218.wav to 16000
converted list1-q9_1664299487431.wav to 16000


In [9]:
tpower = 1.3
tpreemphasis = 0.98
tdb = 20
######################


dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train='metaData_list1_1664477626975..txt', path=os.path.join(output_path, dataset_path)
)

audio_config = BaseAudioConfig(
    sample_rate=22050, 
    win_length=1024, 
    hop_length=256, 
    num_mels=80, 
    mel_fmin=0, 
    mel_fmax=None, 
    power=tpower,
    preemphasis=tpreemphasis,
    ref_level_db=tdb
)

config = VitsConfig(
    audio=audio_config,
    run_name="vits_ljspeech",
    batch_size=32,
    eval_batch_size=16,
    batch_group_size=5,
    num_loader_workers=8,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100,
    text_cleaner="english_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=True,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    cudnn_benchmark=False
)

In [10]:
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    # eval_split_size=config.eval_split_size
    eval_split_size=0.1
)

ap = AudioProcessor.init_from_config(config)
tokenizer, config = TTSTokenizer.init_from_config(config)
model = Vits(config, ap, tokenizer, speaker_manager=None)

 | > Found 10 files in /content/digitalclone-backend/aws_jobs_voiceclone/tts_tests/testdatasets/audiofiles
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.3
 | > preemphasis:0.98
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [11]:
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

trainer.fit()

 > Training Environment:
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False

 > Model has 83059180 parameters

 > EPOCH: 0/100
 --> /content/trainoutput/vits_ljspeech-December-10-2022_11+39PM-0000000


[*] Pre-computing phonemes...


 56%|█████▌    | 5/9 [00:00<00:00, 15.17it/s]

['<BLNK>', 'ə', '<BLNK>', ' ', '<BLNK>', 'l', '<BLNK>', 'ɑ', '<BLNK>', 'ɹ', '<BLNK>', 'd', '<BLNK>', '͡', '<BLNK>', 'ʒ', '<BLNK>', ' ', '<BLNK>', 's', '<BLNK>', 'a', '<BLNK>', 'ɪ', '<BLNK>', 'z', '<BLNK>', ' ', '<BLNK>', 'ɪ', '<BLNK>', 'n', '<BLNK>', ' ', '<BLNK>', 's', '<BLNK>', 't', '<BLNK>', 'ɑ', '<BLNK>', 'k', '<BLNK>', 'ɪ', '<BLNK>', 'ŋ', '<BLNK>', 'z', '<BLNK>', ' ', '<BLNK>', 'ɪ', '<BLNK>', 'z', '<BLNK>', ' ', '<BLNK>', 'h', '<BLNK>', 'ɑ', '<BLNK>', 'ɹ', '<BLNK>', 'd', '<BLNK>', ' ', '<BLNK>', 't', '<BLNK>', 'ə', '<BLNK>', ' ', '<BLNK>', 's', '<BLNK>', 'ɛ', '<BLNK>', 'l', '<BLNK>']
 [!] Character '͡' not found in the vocabulary. Discarding it.


100%|██████████| 9/9 [00:00<00:00, 15.80it/s]
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

 > TRAINING (2022-12-10 23:39:36) 


...

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

 > EVALUATION 





> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
	| > 1 not found characters:
	| > ͡
| > Number of instances : 1
 | > Preprocessing samples
 | > Max text length: 39
 | > Min text length: 39
 | > Avg text length: 39.0
 | 
 | > Max audio length: 71079.0
 | > Min audio length: 71079.0
 | > Avg audio length: 71079.0
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.


   --> STEP: 0
     | > loss_disc: 1.84394  (1.84394)
     | > loss_disc_real_0: 0.08743  (0.08743)
     | > loss_disc_real_1: 0.29799  (0.29799)
     | > loss_disc_real_2: 0.24112  (0.24112)
     | > loss_disc_real_3: 0.20740  (0.20740)
     | > loss_disc_real_4: 0.19234  (0.19234)
     | > loss_disc_real_5: 0.08062  (0.08062)
     | > loss_0: 1.84394  (1.84394)
     | > loss_gen: 2.76738  (2.76738)
     | > loss_kl: 2.03847  (2.03847)
     | > loss_feat: 5.45541  (5.45541)
     | > loss_mel: 38.70301  (38.70301)
     | > loss_duration: 2.26760  (2.26760)
     | > loss_1: 51.23187  (51.23187)



 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.20933 (+0.00419)
     | > avg_loss_disc: 1.84394 (+0.14634)
     | > avg_loss_disc_real_0: 0.08743 (+0.05517)
     | > avg_loss_disc_real_1: 0.29799 (+0.21435)
     | > avg_loss_disc_real_2: 0.24112 (+0.16559)
     | > avg_loss_disc_real_3: 0.20740 (+0.02706)
     | > avg_loss_disc_real_4: 0.19234 (+0.01506)
     | > avg_loss_disc_real_5: 0.08062 (-0.05389)
     | > avg_loss_0: 1.84394 (+0.14634)
     | > avg_loss_gen: 2.76738 (-0.03296)
     | > avg_loss_kl: 2.03847 (+0.61019)
     | > avg_loss_feat: 5.45541 (+0.34282)
     | > avg_loss_mel: 38.70301 (-4.20218)
     | > avg_loss_duration: 2.26760 (-0.09893)
     | > avg_loss_1: 51.23187 (-3.38106)



In [ ]:
## test it!

In [14]:
!tts --text "this is my new voice, lets see how it works. Maybe it does not sound like me." \
      --model_path 'trainoutput/output/best_model_86.pth' \
      --config_path 'trainoutput/output/config.json' \
      --out_path 'trainoutput/output/out.wav'

 > Downloading model to /root/.local/share/tts/vocoder_models--en--ljspeech--univnet
100% 623M/623M [00:14<00:00, 42.6MiB/s]
 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Vo